In [1]:
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import SparsePauliOp
from qiskit import QuantumCircuit

from qiskit.primitives import BackendEstimator,Estimator,Sampler
from qiskit import Aer

from squlearn.util.optree import *
from squlearn.util.optree.optree import *
from squlearn.util.optree.optree_derivative import *
from squlearn.util.optree.optree_evaluate import *

#circuit = random_circuit(4, 2, seed=12)
#circuit.draw()
circuit = QuantumCircuit(4)
circuit.h([0,1,2,3])
circuit.u(0.5,0.5,0.5,[0,1,2,3])
circuit.cx(0,1)
circuit.cx(2,3)
circuit.cx(1,2)

In [2]:
op = SparsePauliOp(["XXXX","IIII"],[0.1,0.2])

In [3]:
tree = OpTreeLeafExpectationValue(circuit,op)
print(tree)


     ┌───┐┌────────────────┐          
q_0: ┤ H ├┤ U(0.5,0.5,0.5) ├──■───────
     ├───┤├────────────────┤┌─┴─┐     
q_1: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├──■──
     ├───┤├────────────────┤└───┘┌─┴─┐
q_2: ┤ H ├┤ U(0.5,0.5,0.5) ├──■──┤ X ├
     ├───┤├────────────────┤┌─┴─┐└───┘
q_3: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├─────
     └───┘└────────────────┘└───┘     

 with observable 
SparsePauliOp(['XXXX', 'IIII'],
              coeffs=[0.1+0.j, 0.2+0.j])



In [4]:
adjust_tree = adjust_measurements(tree,abelian_grouping=False)
print(adjust_tree)

(1.0*
     ┌───┐┌────────────────┐     ┌───┐┌─┐           
q_0: ┤ H ├┤ U(0.5,0.5,0.5) ├──■──┤ H ├┤M├───────────
     ├───┤├────────────────┤┌─┴─┐└───┘└╥┘┌───┐┌─┐   
q_1: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├──■───╫─┤ H ├┤M├───
     ├───┤├────────────────┤└───┘┌─┴─┐ ║ ├───┤└╥┘┌─┐
q_2: ┤ H ├┤ U(0.5,0.5,0.5) ├──■──┤ X ├─╫─┤ H ├─╫─┤M├
     ├───┤├────────────────┤┌─┴─┐├───┤ ║ └┬─┬┘ ║ └╥┘
q_3: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├┤ H ├─╫──┤M├──╫──╫─
     └───┘└────────────────┘└───┘└───┘ ║  └╥┘  ║  ║ 
c: 4/══════════════════════════════════╩═══╩═══╩══╩═
                                       0   3   1  2 

 with observable 
SparsePauliOp(['ZZZZ'],
              coeffs=[0.1+0.j])
 + 1.0*
     ┌───┐┌────────────────┐          ┌─┐
q_0: ┤ H ├┤ U(0.5,0.5,0.5) ├──■───────┤M├
     ├───┤├────────────────┤┌─┴─┐     └╥┘
q_1: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├──■───╫─
     ├───┤├────────────────┤└───┘┌─┴─┐ ║ 
q_2: ┤ H ├┤ U(0.5,0.5,0.5) ├──■──┤ X ├─╫─
     ├───┤├────────────────┤┌─┴─┐└───┘ ║ 
q_3: ┤ H ├┤ U(0.5,0.5,0.5) ├┤ X ├─

In [5]:
#evaluate_expectation_tree_from_sampler(tree,{},Sampler())

In [6]:
evaluate_expectation_tree_from_sampler(adjust_tree,{},Sampler())

build_lists_and_index_tree 0.0
run time 0.007002115249633789
post processing 0.0
expec [0.0198936 0.2      ]


0.2198935958794378

In [7]:
job = Estimator().run(circuit,op)
job.result().values

array([0.2198936])

In [8]:
# Create a backend estimator
backend = Aer.get_backend('statevector_simulator')
estimator = BackendEstimator(backend=backend,options={'shots':1000000})

job = estimator.run(circuit,op)

obs.paulis.x [[ True  True  True  True]
 [False False False False]]
obs.paulis.z [[False False False False]
 [False False False False]]
paulis ['XXXX', 'IIII']
     ┌───┐┌─┐         
q_0: ┤ H ├┤M├─────────
     ├───┤└╥┘┌─┐      
q_1: ┤ H ├─╫─┤M├──────
     ├───┤ ║ └╥┘┌─┐   
q_2: ┤ H ├─╫──╫─┤M├───
     ├───┤ ║  ║ └╥┘┌─┐
q_3: ┤ H ├─╫──╫──╫─┤M├
     └───┘ ║  ║  ║ └╥┘
c: 4/══════╩══╩══╩══╩═
           0  1  2  3 

In [9]:
job.result().values


['XXXX', 'IIII']
[0.1 0.2]
obs.paulis.x [[ True  True  True  True]
 [False False False False]]
obs.paulis.z [[False False False False]
 [False False False False]]
paulis ['XXXX', 'IIII']
     ┌───┐┌─┐         
q_0: ┤ H ├┤M├─────────
     ├───┤└╥┘┌─┐      
q_1: ┤ H ├─╫─┤M├──────
     ├───┤ ║ └╥┘┌─┐   
q_2: ┤ H ├─╫──╫─┤M├───
     ├───┤ ║  ║ └╥┘┌─┐
q_3: ┤ H ├─╫──╫──╫─┤M├
     └───┘ ║  ║  ║ └╥┘
c: 4/══════╩══╩══╩══╩═
           0  1  2  3 
['XXXX', 'IIII']
[0.1 0.2]
paulis ['XXXX', 'IIII']


array([0.2199196])